# Neural Network Basics Tutorial
This notebook demonstrates how to implement and train a basic neural network from scratch using NumPy.

## Contents
1. Implementation of Neural Network
2. Training on a Simple Dataset
3. Visualization of Results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

## 1. Neural Network Implementation

In [ ]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.weights1 = np.random.randn(input_size, hidden_size) * 0.01
        self.weights2 = np.random.randn(hidden_size, output_size) * 0.01
        self.bias1 = np.zeros((1, hidden_size))
        self.bias2 = np.zeros((1, output_size))
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward(self, X):
        self.layer1 = self.sigmoid(np.dot(X, self.weights1) + self.bias1)
        self.output = self.sigmoid(np.dot(self.layer1, self.weights2) + self.bias2)
        return self.output
    
    def backward(self, X, y, learning_rate):
        m = X.shape[0]
        
        # Calculate gradients
        output_error = self.output - y
        output_delta = output_error * self.sigmoid_derivative(self.output)
        
        layer1_error = np.dot(output_delta, self.weights2.T)
        layer1_delta = layer1_error * self.sigmoid_derivative(self.layer1)
        
        # Update weights and biases
        self.weights2 -= learning_rate * np.dot(self.layer1.T, output_delta) / m
        self.weights1 -= learning_rate * np.dot(X.T, layer1_delta) / m
        self.bias2 -= learning_rate * np.sum(output_delta, axis=0, keepdims=True) / m
        self.bias1 -= learning_rate * np.sum(layer1_delta, axis=0, keepdims=True) / m

## 2. Generate and Prepare Data

In [ ]:
# Generate synthetic dataset
X, y = make_classification(n_samples=1000, n_features=2, n_classes=2, n_clusters_per_class=1)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape y to match network output
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

## 3. Train the Network

In [ ]:
# Initialize network
nn = NeuralNetwork(input_size=2, hidden_size=4, output_size=1)

# Training parameters
epochs = 1000
learning_rate = 0.1

# Training loop
losses = []
for epoch in range(epochs):
    # Forward pass
    output = nn.forward(X_train)
    
    # Calculate loss
    loss = np.mean((output - y_train) ** 2)
    losses.append(loss)
    
    # Backward pass
    nn.backward(X_train, y_train, learning_rate)
    
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')

## 4. Visualize Results

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.title('Training Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# Plot decision boundary
def plot_decision_boundary(X, y, model):
    h = 0.02  # Step size in the mesh
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    Z = model.forward(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu, alpha=0.3)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')

plt.figure(figsize=(10, 8))
plot_decision_boundary(X_test, y_test, nn)
plt.title('Decision Boundary')
plt.show()